## What is Parallelism?

## Why would that be useful?


In [1]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt

%matplotlib inline

### Context:

So to show the advantage of parallelism, and make it more clear in our examples what the differences between parallel and serial setups are, we're going to build a couple of monte-carlo integrators - that is numerical approximations of definite integrals via monte-carlo sampling. 


Monte Carlo integration approximates the area under a curve by averaging the heights of many random samples from the curve;
![example](test_func_base.svg)
The more points that are taken, the better the approximation.

### Initial solution (serial)

lets say we want to integrate some function, $f(x)=\text{tanh}\left(x^{\frac{1}{3}} + x^{-3} - \frac{3}{2}x^{-1}\right)$, between $x=0.1$ and $x=2$

In [2]:
def arbitrary_func(x):
    return np.tanh(x**(1/3) + x**(-3) - (3/2)*x**-1)

In [3]:
def get_sample(func, a, b):
    random_pos = np.random.uniform(a,b)
    return func(random_pos)

def monte_carlo_integrate_serial(func, a, b, sample_size=1000):
    samples = [get_sample(func, a, b) for _ in range(sample_size)]
    return (b-a) * sum(samples)/sample_size

print("""
monte carlo solutions -
10 samples:    {}
100 samples:   {}
1000 samples:  {}
10000 samples: {}
""".format(monte_carlo_integrate_serial(arbitrary_func, 0.1, 2, 10),
           monte_carlo_integrate_serial(arbitrary_func, 0.1, 2, 100),
           monte_carlo_integrate_serial(arbitrary_func, 0.1, 2, 1000),
           monte_carlo_integrate_serial(arbitrary_func, 0.1, 2, 10000)
          )
     )

print("scipy's fancy integrator says the answer is about {0[0]:.6g} +- {0[1]:.3g}".format(
    integrate.quadrature(arbitrary_func, 0.1, 2))
     )


monte carlo solutions -
10 samples:    0.8920320635525698
100 samples:   1.2680625801108831
1000 samples:  1.2397725482226427
10000 samples: 1.2333568520980813

scipy's fancy integrator says the answer is about 1.24169 +- 8.91e-09


#### Obviously this isn't optimised anywhere near as well as it could have been - not includung that basic monte carlo integration isn't fantastic in general, we could have used numpy vectorisation to generate all the random samples with a single call to get_sample. Ignoring that, because that isn't the focus of this section, we're going to try to improve it by parallelising it.

#### First, let's say we want to get an accuracy of about 10^-3.

In [16]:
scipy_ans = integrate.quadrature(arbitrary_func, 0.1, 2)[0]
print("""
monte carlo solutions -
error @ 10 samples:       {:.4g}
error @ 100 samples:      {:.4g}
error @ 1000 samples:     {:.4g}
error @ 10000 samples:    {:.4g}
error @ 100000 samples:   {:.4g}
error @ 1000000 samples:  {:.4g}
""".format(monte_carlo_integrate_serial(arbitrary_func, 0.1, 2, 10)-scipy_ans,
           monte_carlo_integrate_serial(arbitrary_func, 0.1, 2, 100)-scipy_ans,
           monte_carlo_integrate_serial(arbitrary_func, 0.1, 2, 1000)-scipy_ans,
           monte_carlo_integrate_serial(arbitrary_func, 0.1, 2, 10000)-scipy_ans,
           monte_carlo_integrate_serial(arbitrary_func, 0.1, 2, 100000)-scipy_ans,
           monte_carlo_integrate_serial(arbitrary_func, 0.1, 2, 1000000)-scipy_ans,
          )
     )


monte carlo solutions -
error @ 10 samples:       -0.2492
error @ 100 samples:      0.0669
error @ 1000 samples:     0.006989
error @ 10000 samples:    0.004879
error @ 100000 samples:   0.0008561
error @ 1000000 samples:  -0.0001551



#### So it looks like we need to take about 2,000,000 samples to get that level of accuracy. But each sample is completely independent from the others - so can we take more than one sample at a time?

In [5]:
from time import time
start = time()
serial_out = monte_carlo_integrate_serial(arbitrary_func, 0.1, 2, 2000000)
end = time()
print(end-start) # baseline

2.1457672119140625e-05


In [19]:
from multiprocessing import Pool, cpu_count

"""
Method one:
"""
sample_size = 2000000
# We need to define a function that generates one sample - we'll adapt that from our serial one
def par_func(x):
    return monte_carlo_integrate_serial(arbitrary_func,0.1,2,1)

p = Pool(processes=cpu_count())

start = time()
par_out = p.map(par_func, range(sample_size))
end = time()

print(np.mean(par_out), end-start)

# Takes ages - we have a lot of work starting new processes, and moving data between the cpu cores

1.2400916018035182 34.43357992172241


In [6]:
from multiprocessing import Pool, cpu_count

"""
Method two:
"""
sample_size = 2000000
batch_size = 100000
# We need to define a function that generates one sample - we'll adapt that from our serial one
def par_func2(x):
    return monte_carlo_integrate_serial(arbitrary_func,0.1,2,batch_size)

p = Pool(processes=cpu_count())

start = time()
par_out = p.map(par_func2, range(sample_size//batch_size))
end = time()

print(np.mean(par_out), end-start)
# Here we get the same thing, but we've reduced the number of times we need to start new processes or move data

1.2420546588264603 8.815419673919678


In [7]:
from joblib import Parallel, delayed

"""
Method three:
"""
start = time()
par_out = Parallel(n_jobs=-1)(
    delayed(
            par_func2
           )(_) 
           for _ in range(sample_size//batch_size))

end = time()

print(np.mean(par_out), end-start)

1.2414749491833201 8.1034414768219


In [14]:

"""
Method four: mpi4py
(see other notebook in this directory)
"""

'\nMethod four: mpi4py\n(see other notebook in this directory)\n'